### Ray UI.

#### To run the UI: 
1. Kernel -> Restart and Run All 


#### To hide the code:

1. Install the hide_code extension.
```
pip install hide_code
jupyter nbextension install --py hide_code
jupyter nbextension enable --py hide_code
jupyter serverextension enable --py hide_code
```
2. Go to View -> Cell Toolbar -> Hide Code. 
3. Press ESC then T

In [ ]:
import ipywidgets as widgets
import os
import ray
import subprocess
import tempfile

from IPython.display import display

ray.init(redis_address=os.environ['REDIS_ADDRESS'])

#### Evaluate the box below to search for objects.

In [ ]:
object_search = widgets.Text(
    value='',
    placeholder='Object ID',
    description='Search for an object:',
    disabled=False
)
display(object_search)

def handle_submit(sender):
    pp = pprint.PrettyPrinter()
    pp.pprint(ray.global_state.object_table(object_search.value))

object_search.on_submit(handle_submit)

#### Evaluate the box below to search for tasks.

In [ ]:
task_search = widgets.Text(
    value='',
    placeholder='Task ID',
    description='Search for a task:',
    disabled=False
)
display(task_search)

def handle_submit(sender):
    pp = pprint.PrettyPrinter()
    pp.pprint(ray.global_state.task_table(task_search.value))

task_search.on_submit(handle_submit)

In [ ]:
def task_timeline():
    path_input = widgets.Button(description='View task timeline')

    breakdown_basic = 'Basic'
    breakdown_task = 'Task Breakdowns'
    
    breakdown_opt = widgets.Dropdown(
        options=['Basic', 'Task Breakdowns'],
        value='Basic',
        description='View options:',
        disabled=False,
    )
    
    start_box, end_box, range_slider, time_opt = get_sliders(False)
    display(breakdown_opt)
    display(path_input)

    def find_trace2html():
        trace2html = '/tmp/ray/catapult/tracing/bin/trace2html'
        # Clone the catapult repository if it doesn't exist. TODO(rkn): We
        # could do this in the build.sh script later on.
        if not os.path.exists(trace2html): 
            cmd = ['git', 'clone', 'https://github.com/catapult-project/catapult.git', '/tmp/ray/catapult']
            subprocess.check_output(cmd) 
            print('Cloning catapult to /tmp/ray/catapult.')
        assert os.path.exists(trace2html)
        return trace2html

    def handle_submit(sender):
        tmp = tempfile.mktemp() + '.json'
        tmp2 = tempfile.mktemp() + '.html'
        
        if breakdown_opt.value == breakdown_basic:
            breakdown = False
        elif breakdown_opt.value == breakdown_task:
            breakdown = True
        else:
            raise ValueError('Unexpected breakdown value "{}"'.format(breakdown_opt.value))
            
        low, high = map(lambda x: x / 100., range_slider.value)
        
        smallest, largest, num_tasks = ray.global_state._job_length()
        diff = largest - smallest

        if time_opt.value == total_time_value:
            tasks = ray.global_state.task_profiles(start=smallest + diff * low, end=smallest + diff * high)
        elif time_opt.value == total_tasks_value:
            if range_slider.value[0] == 0:
                tasks = ray.global_state.task_profiles(num_tasks=int(num_tasks * high), fwd=True)
            else:
                tasks = ray.global_state.task_profiles(num_tasks=int(num_tasks * (high - low)), fwd=False)
        else:
            raise ValueError('Unexpected time value "{}"'.format(time_opt.value))
        
        print('{} tasks to trace'.format(len(tasks)))
        print('Dumping task profiling data to ' + tmp)
        ray.global_state.dump_catapult_trace(tmp, tasks, breakdowns=breakdown)
        print('Converting chrome trace to ' + tmp2)
        trace2html = find_trace2html()
        # TODO(rkn): The trace2html script currently requires Python 2.
        # Remove this dependency.
        subprocess.check_output(['python2', trace2html, tmp, '--output', tmp2])
        # Open the timeline in Chrome. TODO(rkn): We should remove the
        # dependency on Chrome and use whatever browser is currently being
        # used. Note that this currently does not work when Ray is being
        # used on a cluster and the browser is running locally.
        print('Opening html file in browser...')
        subprocess.Popen(['open', '-a', 'Google Chrome', tmp2])

    path_input.on_click(handle_submit)

task_timeline()